# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train) & set(users_test))

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


In [5]:
cold_users = list(set(users_test) - set(users_train))

print(len(cold_users))

2365


In [6]:
events_train

,user_id,count_read_book,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
0,1000000,29,27774758,2017-01-02,2017-02-02,True,5,False,2017-01-01
1,1000000,29,16101128,2016-09-17,2016-10-05,True,3,False,2016-09-01
2,1000000,29,18798983,2017-03-26,2017-05-12,True,3,False,2017-03-01
3,1000000,29,9969571,2016-08-26,2016-09-15,True,4,False,2016-08-01
4,1000000,29,17851885,2016-08-01,2016-08-09,True,4,False,2016-08-01
...,...,...,...,...,...,...,...,...,...
11751081,1430584,13,18243700,2016-04-29,2016-05-09,True,4,True,2016-04-01
11751082,1430584,13,6614960,2015-11-02,2015-12-25,True,3,False,2015-11-01
11751083,1430584,13,24817626,2015-11-02,2015-11-02,True,1,False,2015-11-01
11751084,1430584,13,7445,2016-04-24,2016-05-18,True,4,True,2016-04-01


In [7]:
from sklearn.preprocessing import MinMaxScaler

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["book_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values(by = ["popularity_score"], ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity['avg_rating']>=4].head(100)

In [8]:
item_popularity

,book_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
35011,22557272,40690,3.788965,1.000000,0.697241,0.697241
37701,29056083,25785,3.801784,0.633685,0.700446,0.443862
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
...,...,...,...,...,...,...
25678,9643433,1,4.000000,0.000000,0.750000,0.000000
25680,9644001,1,3.000000,0.000000,0.500000,0.000000
25684,9647896,1,5.000000,0.000000,1.000000,0.000000
25692,9654320,1,4.000000,0.000000,0.750000,0.000000


In [9]:
top_k_pop_items

,book_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076
...,...,...,...,...,...,...
24837,8490112,4792,4.080968,0.117747,0.770242,0.090694
33611,18966819,4361,4.374914,0.107154,0.843729,0.090409
378,3636,4667,4.098564,0.114675,0.774641,0.088832
32835,18293427,4674,4.092640,0.114847,0.773160,0.088795


In [10]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("book_id")[["author", "title", "genre_and_votes", "publication_year"]], on="book_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["book_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,book_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [11]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="book_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "book_id", "rating", "avg_rating"]]

In [12]:
events_test

,user_id,count_read_book,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month
35,1000003,95,25893709,2017-10-05,2017-10-17,True,4,False,2017-10-01
263,1000005,17,34076952,2017-10-09,2017-10-24,True,5,False,2017-10-01
282,1000006,22,24817626,2017-09-17,2017-10-20,True,3,False,2017-09-01
284,1000006,22,18812405,2017-08-05,2017-08-19,True,3,False,2017-08-01
290,1000006,22,18774964,2017-10-04,2017-10-22,True,4,False,2017-10-01
...,...,...,...,...,...,...,...,...,...
11750818,1430579,265,27272506,2017-09-03,2017-10-07,True,3,True,2017-09-01
11751057,1430580,3,15749186,2017-10-05,2017-10-18,True,4,False,2017-10-01
11751058,1430580,3,22021611,2017-10-05,2017-10-05,True,4,False,2017-10-01
11751073,1430584,13,18692431,2017-08-02,2017-08-09,True,3,True,2017-08-01


In [13]:
cold_users_events_with_recs['popularity_score'].isna().sum()/cold_users_events_with_recs.shape[0]

0.8023159636062862

In [14]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [15]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

In [16]:
events[['user_id', 'book_id', 'rating']].shape[0]


11751086

In [17]:
events.query('rating == 0')

,user_id,count_read_book,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month


In [18]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'book_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [19]:
surprise_test_set = list(events_test[['user_id', 'book_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [20]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8275
MAE:  0.6469
0.8274973520335469 0.6468999661665751


In [21]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [22]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)
                     
print(rmse, mae) 

RMSE: 1.2597
MAE:  0.9987
1.259733700763627 0.9987421589285332


In [25]:
events[events['user_id']==1000100]['book_id']

3382    13539044
3383    13538873
3384    13083008
3385    10818853
3386     1044355
3387    10407279
Name: book_id, dtype: int64

In [35]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=10):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['book_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id']==user_id]['book_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, book_id) for book_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["book_id", "score"])

In [36]:
get_recommendations_svd(1296647, items, events_test, svd_model)

,book_id,score
0,481749,5.000000
1,18713259,4.984556
2,10463536,4.951878
3,862041,4.920939
4,33353628,4.920885
5,323355,4.916639
6,580213,4.885675
7,25667474,4.876574
8,11221285,4.873091
9,30688013,4.871236


In [38]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("book_id")[["author", "title", "genre_and_votes"]], on="book_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["book_id", "author", "title", "genre_and_votes"]], on="book_id")
display(user_recommendations)

user_id: 1312654
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
122,"P.C. Cast, Kristin Cast","Hidden (House of Night, #10)",2015-02-08,2015-02-10,3,"{'Paranormal-Vampires': 683, 'Young Adult': 66..."
123,"P.C. Cast, Kristin Cast","Destined (House of Night, #9)",2014-10-21,2014-10-22,3,"{'Paranormal-Vampires': 956, 'Young Adult': 93..."
124,Kerstin Gier,"Smaragdgrün (Edelstein-Trilogie, #3)",2016-08-31,2016-09-01,5,"{'Fantasy': 1818, 'Young Adult': 1349, 'Scienc..."
125,Kerstin Gier,"Saphirblau (Edelstein-Trilogie, #2)",2016-08-31,2016-08-31,5,"{'Fantasy': 2031, 'Young Adult': 1546, 'Scienc..."
126,Kerstin Gier,"Rubinrot (Edelstein-Trilogie, #1)",2016-08-30,2016-08-31,5,"{'Fantasy': 3141, 'Young Adult': 2505, 'Scienc..."
127,Stephenie Meyer,The Short Second Life of Bree Tanner (Twilight...,2015-04-04,2015-04-06,4,"{'Young Adult': 2775, 'Fantasy': 2380, 'Parano..."
128,Stephenie Meyer,"Breaking Dawn (Twilight, #4)",2017-02-12,2017-02-13,5,"{'Young Adult': 13065, 'Fantasy': 11631, 'Roma..."
129,Stephenie Meyer,"Eclipse (Twilight, #3)",2017-02-10,2017-02-11,5,"{'Young Adult': 13766, 'Fantasy': 12085, 'Roma..."
130,"Stephenie Meyer, Luc Rigoureau","New Moon (Twilight, #2)",2017-02-08,2017-02-09,5,"{'Young Adult': 14292, 'Fantasy': 12580, 'Roma..."
131,Stephenie Meyer,"Twilight (Twilight, #1)",2017-02-02,2017-02-07,5,"{'Young Adult': 19035, 'Fantasy': 17639, 'Roma..."


Рекомендации


,book_id,score,author,title,genre_and_votes
0,10,5,J.K. Rowling,"Harry Potter Collection (Harry Potter, #1-6)","{'Fantasy': 282, 'Fiction': 96, 'Young Adult':..."
1,15729392,5,Laia Soler,Los días que nos separan,"{'Romance': 29, 'Young Adult': 23, 'European L..."
2,1049531,5,Andrew D. Olsen,The Price We Paid: The Extraordinary Story of ...,"{'Nonfiction': 33, 'History': 30, 'Religion': ..."
3,1111,5,Robert A. Caro,The Power Broker: Robert Moses and the Fall of...,"{'History': 793, 'Biography': 716, 'Nonfiction..."
4,13633498,5,Ramlee Awang Murshid,"Sunan Musafir (Bagaikan Puteri, #6)",{'Novels': 5}
5,7342071,5,"Juan Díaz Canales, Juanjo Guarnido, Anthya Flo...","Blacksad (Blacksad, #1-3)","{'Sequential Art-Graphic Novels': 627, 'Sequen..."
6,17828703,5,Eva Schloss,After Auschwitz,"{'Nonfiction': 55, 'History': 39, 'World War I..."
7,32903079,5,Christelle Dabos,La Mémoire de Babel,"{'Fantasy': 461, 'Young Adult': 96, 'Cultural-..."
8,20451634,5,Inio Asano,Oyasumi Punpun,"{'Sequential Art-Manga': 248, 'Sequential Art-..."
9,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."


# === Базовые подходы: коллаборативная фильтрация

In [41]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["book_id"])
items["book_id_enc"] = item_encoder.transform(items["book_id"])
events_train["book_id_enc"] = item_encoder.transform(events_train["book_id"])
events_test["book_id_enc"] = item_encoder.transform(events_test["book_id"])

/tmp/ipykernel_2124/3388831299.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_2124/3388831299.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_2124/3388831299.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [45]:
events_train['book_id_enc'].max()

43304

In [59]:
a = events['book_id'].nunique() * events['user_id'].nunique() / (1024**3) 
a 

16.71143686864525

In [61]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['book_id_enc'])),
    dtype=np.int8)

In [62]:
import sys

sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

In [63]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [02:55<00:00,  3.51s/it]


In [64]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"book_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["book_id"] = item_encoder.inverse_transform(recommendations["book_id_enc"])
    
    return recommendations

In [65]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [66]:
# преобразуем полученные рекомендации в табличный формат
book_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "book_id_enc": book_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["book_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["book_id_enc"] = als_recommendations["book_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["book_id"] = item_encoder.inverse_transform(als_recommendations["book_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "book_id_enc"])

In [67]:
als_recommendations = als_recommendations[["user_id", "book_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [69]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "book_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "book_id"], how="left")
)

In [70]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

In [71]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [72]:
print(ndcg_at_5_scores.mean())

0.975946709792109


# === Базовые подходы: контентные рекомендации

In [76]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [77]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes  
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [78]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [79]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [ ]:
items = items.sort_values(by="book_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210895 stored elements and shape (43312, 815)>

In [113]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["book_id", "rating"]]
user_items = items[items["book_id"].isin(user_events["book_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [116]:
user_events

,book_id,rating
464,18143977,5
465,26228034,5
466,20510241,4
467,24817626,2
468,16299,3
469,19161852,5
470,80660,4
471,7331435,4
472,29044,4
473,12262741,5


In [117]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [118]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.185241
38,Classics,3414934,0.103879
25,Fantasy,6850060,0.072447
5,Nonfiction,1737406,0.050865
24,Science Fiction,1218917,0.040920


In [120]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[::-1][:k]

In [122]:
selected_items = items[items["book_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"
39408,"Paulo Coelho, Alan R. Clarke, James Noel Smith",The Alchemist,"{'Fiction': 14023, 'Classics': 5787, 'Fantasy': 3289, 'Philosophy': 2759}"


# === Базовые подходы: валидация

# === Двухстадийный подход: метрики

# === Двухстадийный подход: модель

# === Двухстадийный подход: построение признаков